In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents.base import Document
from langgraph.graph.message import add_messages
import dotenv
import os

dotenv.load_dotenv()

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

file_path = "./files/"
file_list = os.listdir(file_path)

In [ ]:
# 리트리버 생성

def embedding_file(file_path):
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=300,
        chunk_overlap=50,
        separators=["\n\n"],
    )

    loader = PyPDFLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(docs, embeddings)
    retriever = vector_store.as_retriever()

    return retriever

retriever = embedding_file(file_path+file_list[1])

In [ ]:
# 그래프 상태 정의

class State(TypedDict):
    messages : Annotated[list, add_messages]
    document : Annotated[Document, "Retrieve Response"]

In [ ]:
# 도큐먼트 생성

def create_document(state:State):
    docs = retriever.invoke(state["messages"][-1].content)

    docs = "\n\n".join(doc.page_content for doc in docs)

    return {"document":docs}

In [ ]:
# 응답 생성성

def response(state:State):
    prompt = ChatPromptTemplate.from_messages([ ### [프롬프트를 입력해주세요] ### ])
    
    chain = # 체인을 만들어주세요.

    return {"messages": chain.invoke({"context" : state["document"],
                                    "query": state["messages"][-1]})}

In [ ]:
# 그래프 빌더 생성

graph_builder = StateGraph(State)

In [ ]:
# 노드 및 엣지 추가

graph_builder.add_node("create_document", create_document);
graph_builder.add_node("response", response);

graph_builder.add_edge(START, "create_document");
graph_builder.add_edge("create_document", "response");
graph_builder.add_edge("response", END); 

In [ ]:
# 그래프 컴파일

graph = graph_builder.compile()

In [ ]:
# 그래프 시각화

graph

In [ ]:
result = graph.invoke({"messages":("user","언제 개강하나요?")})

In [ ]:
result["messages"][-1].content

In [ ]:
result = graph.invoke({"messages":("user","오프라인 강의인가요?")})

In [ ]:
result["messages"][-1].content

In [ ]:
result = graph.invoke({"messages":("user","졸업하면 어떤걸 할 수 있나요?")})

In [ ]:
result["messages"][-1].content

In [ ]:
result = graph.invoke({"messages":("user","서울에서 부산까지는 얼마나 걸리나요?")})

In [ ]:
result

In [ ]:
result["messages"][-1].content